In [1]:
import numpy as np
from scipy.optimize import minimize, root_scalar

# Define parameters
mu_G = 1  # weight for global surplus
mu_L = 1  # weight for local surplus
Q_G = 100  # global quantity
Q_L = 100  # local quantity
epsilon_DG = 1.5  # global elasticity
epsilon_DL = 2.5  # local elasticity

# Function to solve for p using the market clearing condition
def solve_p(t, tau_g):
    def market_clearing(p):
        theta_tilde = p * (1 - tau_g) / (1 - t)
        lhs = theta_tilde * (
            1
            + ((1 - 1 / epsilon_DL) ** epsilon_DL * Q_L)
            / (p ** epsilon_DL * (epsilon_DL - 1))
        )
        rhs = Q_G * p ** (-epsilon_DG)
        return lhs - rhs

    # Solve for p using a numerical root-finding method
    result = root_scalar(market_clearing, bracket=[1e-3, 100], method='brentq')
    return result.root if result.converged else None

# Reformulate the objective function
def revised_objective(params):
    t, tau_g = params
    p = solve_p(t, tau_g)
    if p is None:  # If p cannot be solved, penalize this solution heavily
        return 1e6
    term_global = mu_G * Q_G / (epsilon_DG - 1) * p ** (1 - epsilon_DG)
    theta_tilde = p * (1 - tau_g) / (1 - t)
    term_local = (
        mu_L
        * Q_L
        * theta_tilde
        / ((epsilon_DL - 1) * (2 - epsilon_DL))
        * ((1 - 1 / epsilon_DL) / p) ** (epsilon_DL - 1)
    )
    return -(term_global + term_local)  # Negative for minimization

# Reformulate the budget constraint
def revised_budget_constraint(params):
    t, tau_g = params
    p = solve_p(t, tau_g)
    if p is None:  # If p cannot be solved, penalize this solution heavily
        return 1e6
    theta_tilde = p * (1 - tau_g) / (1 - t)
    lhs = (
        p * (1 - tau_g) / (1 - t)
        * tau_g
        * Q_L
        / (p ** (epsilon_DL - 1) * (2 - epsilon_DL))
        * (1 - 1 / epsilon_DL) ** (epsilon_DL - 1)
    )
    rhs = t * (theta_tilde ** 2) * (
        1 / 2
        - ((1 - 1 / epsilon_DL) ** epsilon_DL * Q_L)
        / (p ** epsilon_DL * (1 - epsilon_DL) * (2 - epsilon_DL))
    )
    return lhs - rhs

# Initial guesses for t and tau_g
initial_guess_revised = [0.1, 0.1]

# Bounds for the variables t and tau_g
bounds_revised = [(0, 1), (0, 1)]  # t and tau_g in [0, 1]

# Solve the revised problem
solution_revised = minimize(
    revised_objective, initial_guess_revised, bounds=bounds_revised,
    constraints=[
        {"type": "eq", "fun": revised_budget_constraint},
    ], method="SLSQP"
)

# Extract the optimal values for t and tau_g
optimal_t = solution_revised.x[0]
optimal_tau_g = solution_revised.x[1]

# Solve for the optimal p
optimal_p = solve_p(optimal_t, optimal_tau_g)

# Compute the cutoff transportation cost (theta_tilde)
optimal_theta_tilde = optimal_p * (1 - optimal_tau_g) / (1 - optimal_t)

# Print the results
print("Optimal Subsidy Rate (t):", optimal_t)
print("Optimal Global Tax Rate (tau_g):", optimal_tau_g)
print("Optimal Transportation Cost Cutoff (theta_tilde):", optimal_theta_tilde)
print("Optimal Price (p):", optimal_p)


Optimal Subsidy Rate (t): 0.0
Optimal Global Tax Rate (tau_g): 4.404571325722362e-20
Optimal Transportation Cost Cutoff (theta_tilde): 5.811261504057678
Optimal Price (p): 5.811261504057678
